In [2]:
from detector import Detector
from detector_lpr import LicensePlateReader

detector = Detector()
detector_lpr = LicensePlateReader(r"D:\omer\parkingEntrance\trained_models\best_v8.pt")

def read_license_plate_number(img, char_threshold):
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('img', 1920, 1080)
    plate_reading = 'NA'
    
    # detect plate
    bboxes = detector.detect(img)
    
    # filter 'plate only' detections
    plate_only = []
    for i in bboxes:
        if i[4] == 'plate':
            plate_only.append(i)
    if not plate_only:
        print('no plate detected in img')
    if plate_only:
        max_tuple = max(plate_only, key=lambda x:x[5].item()) # find the most probable plate detection
        x1 = max_tuple[0]
        y1 = max_tuple[1]
        x2 = max_tuple[2]
        y2 = max_tuple[3]
        plate_crop = img[y1:y2, x1:x2]
        # cv2.imshow('img', img)
        # cv2.waitKey(0)
        # cv2.imshow('img', plate_crop)
        # cv2.waitKey(0)
        return plate_crop

YOLOv5  2023-3-26 Python-3.11.8 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7268094 parameters, 0 gradients


In [12]:
import glob
import cv2
root = r"D:\omer\vehicleClassification\entryImages"
image_files = glob.glob(root + "/*.jpg")
print(image_files)

for image_path in image_files:
    file_parts = image_path.split('\\')[-1]
    file_name_parts = file_parts.split('_')
    vehicleSize = file_name_parts[1]
    entryTime = file_name_parts[2].split('.')[0]  # Removing the file extension

    img = cv2.imread(image_path)
    plate_crop = read_license_plate_number(img, 0.5)
    try:
        plate_reading = detector_lpr.detect(plate_crop)
    except Exception as e:
        print('could not lpr, reason: ', e)
    

[]


In [3]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime

def generate_parking_ticket(vehicle_size, entry_time, plate_reading, image_path):
    # Configure PDF file name and location
    ticket_file_name = f"D:\\omer\\parkingEntrance\\tickets\\Parking_Ticket_{plate_reading}.pdf"
    
    # Create a PDF canvas
    c = canvas.Canvas(ticket_file_name, pagesize=letter)
    width, height = letter  # Get the dimensions of the page
    
    # Add title
    c.setFont("Helvetica-Bold", 35)
    c.drawString(50, height - 100, "Parking Ticket")
    
    # Vehicle Size
    c.setFont("Helvetica", 20)
    c.drawString(50, height - 140, f"Vehicle Size: {vehicle_size}")
    
    # Entry Time
    c.drawString(50, height - 160, f"Entry Time: {entry_time}")
    
    # Plate Reading
    c.drawString(50, height - 180, f"License Plate: {plate_reading}")
    
    # Vehicle Image
    # Assuming the image is not too large and fits within the page. You might need to resize it.
    c.drawImage(image_path, 50, height - 350, width=150, height=150, preserveAspectRatio=True, anchor='nw')
    
    # Save the PDF
    c.showPage()
    c.save()
    print(f"Parking ticket generated: {ticket_file_name}")

# Example usage
# generate_parking_ticket("xx", "xx", "xxx", r"D:\omer\vehicleClassification\entryImages\frame1_small_12-21.jpg")


In [7]:
import os
import time
from datetime import datetime
from IPython.display import clear_output
import glob
import cv2

root = r"D:\omer\vehicleClassification\entryImages"
processed_images = []

while True:
    image_files = glob.glob(root + "/*.jpg")
    
    if len(image_files) > 0:
        for image_path in image_files:
            if image_path in processed_images:
                continue
            
            file_parts = image_path.split('\\')[-1]
            file_name_parts = file_parts.split('_')
            vehicleSize = file_name_parts[1]
            entryTime = file_name_parts[2].split('.')[0]  # Removing the file extension
            today = datetime.now().strftime("%Y-%m-%d")
            entryTime = today + " " + entryTime.replace('-', ':')

            img = cv2.imread(image_path)
            plate_crop = read_license_plate_number(img, 0.5)
            try:
                plate_reading = detector_lpr.detect(plate_crop)
            except Exception as e:
                print('could not lpr, reason: ', e)

            generate_parking_ticket(vehicleSize, entryTime, plate_reading, image_path)
            ticket_file_name = f"D:\\omer\\parkingEntrance\\tickets\\Parking_Ticket_{plate_reading}.pdf"
            print(f"Printing parking ticket: {ticket_file_name}")
            # print_pdf_windows(ticket_file_name, printer_name=r"\\prn01\Hp_q_eelab2")
            
            processed_images.append(image_path)  # Add the processed image path to the list
        
          
    print("waiting 2 secs before checking again")   
    time.sleep(2)  # Wait for 5 seconds before checking again
    clear_output(wait=True)
    


0: 288x640 2 0s, 1 5, 1 A, 1 R, 1 X, 14.0ms
Speed: 4.3ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)
results of yolo_lpr:  6
final_plate_str:  ARX500
Parking ticket generated: D:\omer\parkingEntrance\tickets\Parking_Ticket_ARX500.pdf
Printing parking ticket: D:\omer\parkingEntrance\tickets\Parking_Ticket_ARX500.pdf
Print command executed. Please check your printer.
waiting 2 secs before checking again


KeyboardInterrupt: 

: 

In [2]:
import glob
root = r"D:\omer\vehicleClassification\entryImages"
image_files = glob.glob(root + "/*.jpg")
print(image_files)


['D:\\omer\\vehicleClassification\\entryImages\\frame1_medium_12-11.jpg', 'D:\\omer\\vehicleClassification\\entryImages\\frame2_small_12-11.jpg', 'D:\\omer\\vehicleClassification\\entryImages\\frame3_small_12-12.jpg']


In [4]:
import win32api
import win32print
import win32con

def print_pdf_windows(pdf_file_path, printer_name=None):
    try:
        if printer_name is None:
            printer_name = win32print.GetDefaultPrinter()
        # Attempt to print the PDF
        win32api.ShellExecute(
            0,
            "printTo",
            pdf_file_path,
            f'"{printer_name}"',
            ".",
            win32con.SW_HIDE,
        )
        print("Print command executed. Please check your printer.")
    except Exception as e:
        print(f"Failed to print: {e}")

# Example usage
# pdf_file_path = r"D:\omer\parkingEntrance\tickets\Parking_Ticket_xx.pdf"  # Update this to your PDF file path
# print_pdf_windows(pdf_file_path, printer_name=r"\\prn01\Hp_q_eelab2")
